# Create custom dataset

CIFAR (foreground) on ImageNet (background)

Size of the output images: **32x32**


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
path = ('/content/drive/Shareddrives/CSCI566-Project')

In [4]:
# gpu_info = !nvidia-smi -i 0
# gpu_info = '\n'.join(gpu_info)
# print(gpu_info)

from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import numpy as np
from IPython.display import Image as Im
import random

In [38]:
def copypaste(cifar_img):

    cifar_size = (16, 16)
    inet_size = (32, 32)

    imagenet_path = path + '/tiny-imagenet-200/images/'
    tiny_imagenet = random.choice(os.listdir(imagenet_path))
    image = Image.open(imagenet_path + tiny_imagenet)

    
    image = image.resize(inet_size)
    cifar_img = cifar_img.resize(cifar_size)

    max_loc = inet_size[0] - cifar_size[0]
    x = random.randint(0, max_loc)
    y = random.randint(0, max_loc)

    x1, y1, x2, y2 = x, y, x + cifar_size[0], y + cifar_size[0]
    image.paste(cifar_img, (x1, y1, x2, y2)) #, mask = cif)

    return np.asarray(image)
    
# cifar = f'{path}/cifar_single_image.jpeg'
# cifar = Image.open(cifar)
# cifar = np.asarray(cifar)
# augmented = copypaste(cifar)

In [6]:
transform = transforms.Compose([copypaste])

cifar_loader = CIFAR10(root='data', train=True, transform=transform, download=True)
cifar_loader = DataLoader(cifar_loader, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


In [35]:
def create_dataset(dataset_size):
  """Creates jped files in the format <labelindex_number.jpeg> e.g. 9_00001.jpg"""
  
  transform = transforms.ToPILImage()

  # create new directory for the new custom images
  dataset_path = path + '/custom_dataset'
  if not os.path.exists(dataset_path):
    os.mkdir(dataset_path)

  # check if the directory is empty
  contents = [file for file in os.listdir(dataset_path) if file.endswith(".jpeg")]
  if len(contents) != 0:
    print("The directory already contains image data. Empty directory before creating new data.")
    return

  num_chars = len(str(dataset_size))
  i = 0
  for cifar_img, target in cifar_loader:
    img = transform(cifar_img[0].permute(2, 0, 1))
    zero_padded_i = str(i + 1).rjust(num_chars, '0')
    file_path = f'{path}/custom_dataset/{target.item()}_{zero_padded_i}.jpeg' 
    img.save(file_path)
    i += 1
    if i >= dataset_size:
      print("Dataset created successfully!")
      return 


In [39]:
dataset_size=10
create_dataset(dataset_size)

Dataset created successfully!


In [37]:
class CopyPasteDataset(Dataset):

  def __init__(
        self,
        root_dir: str,
        transform=transform,
        train: bool = True,   # optional if we decide to separate dataset into train and test
    ):
    self.transform = transform

    self.data = []
    self.labels = []

    for filename in os.listdir(root_dir):
      if filename.endswith(".jpeg"):
        image = Image.open(f'{root_dir}/{filename}')

        data = np.asarray(image)

        label = int(filename.split('_')[0])
      
        self.data.append(data)
        self.labels.append(label)

  def __len__(self):
      return len(self.data)

  def __getitem__(self, index: int):

      img, target = self.data[index], self.labels[index]

      img = Image.fromarray(img)

      if self.transform is not None:
        img = self.transform(img)

      return img, target


In [176]:
transform = transforms.Compose([             
    transforms.ToTensor()])

cp_loader = CopyPasteDataset(root_dir=f'{path}/custom_dataset', transform=transform)
cp_loader = DataLoader(cp_loader, batch_size=4, shuffle=True, num_workers=1)

for img, target in cp_loader:
  print(target)

tensor([4, 7, 3, 4])
tensor([1, 2, 8, 6])
tensor([9, 1, 9])
